In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

: 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import os
from sklearn.model_selection import cross_validate
from math import pi
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

: 

In [ ]:
df = pd.read_csv('../raw_data/datos.csv', delimiter = ";")

: 

In [ ]:
df

: 

In [ ]:
app = FastAPI()

: 

In [ ]:
# Define a root `/` endpoint
@app.get('/')
def index():
    return {'greeting': 'Hello'}

: 

In [ ]:
DTYPES_RAW_OPTIMIZED = {
    "key": "O",
    "ContadorBajasCCC": "int64",
    "ContadorBajasDNI": "int64",
    "sexo": "int64",
    "cnae": "int64",
    "icd9": "float64",
    "recaida": "int64",
    "numtreb": "int64",
    "codipostal": "int64",
    "ContadordiasBajasDNI": "int64",
    "contracte": "int64",
    "grupcoti": "int64",
    "pluriempleo": "int64",
    "diasemana": "int64",
    "tiempo_en_empresa": "float64",
    "edad": "float64",
    "mes_baja": "int64",
    "epiweek": "int64"
}


: 

In [ ]:
COLUMN_NAMES_RAW = DTYPES_RAW_OPTIMIZED.keys()

: 

In [ ]:
COLUMN_NAMES_RAW

: 

In [ ]:
# Define a prediction `/` endpoint
@app.get('/predict')
def predict(ContadorBajasCCC: int,
            ContadorBajasDNI: int,
            sexo: int,
            cnae: int,
            icd9: float,
            recaida: int,
            numtreb: int,
            codipostal: int,
            ContadordiasBajasDNI: int,
            contracte: int,
            grupcoti: int,
            pluriempleo: int,
            diasemana: int,
            tiempo_en_empresa: float,
            edad: float,
            mes_baja: int,
            epiweek: int):
    """
    we use type hinting to indicate the data types expected
    for the parameters of the function
    FastAPI uses this information in order to hand errors
    to the developpers providing incompatible parameters
    FastAPI also provides variables of the expected data type to use
    without type hinting we need to manually convert
    the parameters of the functions which are all received as strings
    """
    key = ContadorBajasCCC
    X_new = pd.DataFrame([key, ContadorBajasCCC, ContadorBajasDNI, sexo, cnae, icd9,
       recaida, numtreb, codipostal, ContadordiasBajasDNI,
       contracte, grupcoti, pluriempleo, diasemana,
       tiempo_en_empresa, edad, mes_baja, epiweek]).T
    X_new.columns = COLUMN_NAMES_RAW
    X_new = X_new.astype(DTYPES_RAW_OPTIMIZED)
    # X_new = preprocess_features(X_new)
    y_pred = float(model.predict(X_new))
    
    return {'fare_amount': y_pred}

: 

In [ ]:
df = pd.read_csv('~/code/jaimesoroa/smarthealing/raw_data/datos.csv', delimiter = ";")
df = df.drop_duplicates()
index1 = df[ (df['numtreb'] == 0)].index
df.drop(index1 , inplace=True)
index2 = df[  (df['contracte'] == '-') ].index
df.drop(index2 , inplace=True)
index3 = df[ (df['cnae'] == 0) ].index
df.drop(index3 , inplace=True)
index4 = df[  (df['codipostal'] == '-')].index
df.drop(index4 , inplace=True)
index5 = df[ (df['tiempo_en_empresa'] == 0.00)].index
df.drop(index5 , inplace=True)
index6 = df[ (df['grupcoti'] == '-')].index
df.drop(index6 , inplace=True)
df_2 = df.sort_values('icd9').head(50000)
df_2['icd9'] = pd.to_numeric(df_2['icd9'])
df_2['codipostal'] = pd.to_numeric(df_2['codipostal'])
df_2['contracte'] = pd.to_numeric(df_2['contracte'])
df_2['grupcoti'] = pd.to_numeric(df_2['grupcoti'])
X = df_2.drop(columns = ['duracion_baja'])
y = df_2["duracion_baja"]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train = X.head(30000)
y_train = y.head(30000)
X_test = X.tail(20000)

: 

In [ ]:
model = LinearRegression()
model_fit = model.fit(X_train, y_train)

: 

In [ ]:
pred_2 = model_fit.predict(X_test.iloc[[0]])
pred_2

: 

In [ ]:
X_test.iloc[[0]]

: 

In [ ]:
pred_3 = model_fit.predict(X_test.iloc[[50]])
pred_3

: 

In [ ]:
X_test.iloc[[50]]

: 

In [ ]:
# 0. Load Data and Drop Duplicates
data = pd.read_csv('../raw_data/datos.csv', delimiter = ';', low_memory = False)
data.drop_duplicates(inplace = True)
outliers = data.query('duracion_baja >= 250')  # Save outliers for analysis.
data = data.query('duracion_baja < 250')
data.drop_duplicates(inplace = True)

# 1. Handle Null Values
# CNAE (only 482)
data = data.query('cnae != 0')

# CodiPostal (Most Frequent)
data['codipostal'] = data['codipostal'].replace('-', (data['codipostal']).mode()[0])

# Contracte (Delete few wrongs and then Most Frequent)
data = data[data['contracte'] != '001']
data = data[data['contracte'] != '019']
data = data[data['contracte'] != '999']

mf_contracte = data['contracte'].mode()[0]
data['contracte'] = data['contracte'].replace('000', mf_contracte).replace('-', mf_contracte)

# Grupcoti (Most Frequent)
mf_grupcoti = data['grupcoti'].mode()[0]
data['grupcoti'] = data['grupcoti'].replace('-', mf_grupcoti).replace('0', mf_grupcoti)

# 2. Data Cleaning
# ContadorBajasCCC (non-sense)
data.drop('ContadorBajasCCC', axis = 1, inplace = True)

# ContadorBajasDNI vs ContadordiasBajasDNI
data = data.query('ContadorBajasDNI <= ContadordiasBajasDNI')

# Categorize CNAE
c = pd.read_excel('Tabla_cnae.xlsx', usecols = ['GRUPO', 'COD_CNAE2009'])
c.rename(columns = {'GRUPO': 'cnae_cat', 'COD_CNAE2009': 'cnae'}, inplace = True)
c['cnae_cat'] = c['cnae_cat'].map(lambda x: ord(x) - 64)
c = c.query('cnae.str.isnumeric()').astype('int')
data = data.merge(c, how = 'left', on = 'cnae')

# Sickness Types
block_ends = [140,240,280,290,320,390,460,520,580,630,680,710,740,760,780,800,1000]
blocks = [[i+1, s] for i, s in enumerate(block_ends)]
special_cases = ['E','V','M']

def make_icd9_cat(row):
    first_str = row.icd9[0]
    if first_str == 'E':
        return 18
    if first_str == 'V':
        return 19
    if first_str == 'M':
        return 20
    else:
        code = float(row.icd9)
        for block in blocks:
            if code < block[1]:
                return block[0]
data['icd9_cat'] = data.apply(lambda row: make_icd9_cat(row), axis=1)

# Sickness Wrong Values
data = data.query('sexo != 1 | icd9_cat != 11')
data = data[~((data['sexo'] == 1) & (data['icd9'].str[:2].isin(['61', '62'])))]
data = data[~((data['sexo'] == 2) & (data['icd9'].str[:2] == '60'))]

# Post Code Categories.
data['codipostal_cat'] = data['codipostal'].str[:2]

# Time in company
# Add 0.01 to have the upper limit. (0.00 means up to 3 days)
data['proporcion_baja'] = (data['duracion_baja'] / (365 * (data['tiempo_en_empresa'] + 0.01)))
by_wrong_proportion = data.sort_values(by = 'proporcion_baja', axis = 0, ascending = False)
data = data.query('proporcion_baja <= 1')
data.drop('proporcion_baja', axis = 1, inplace = True)

# Month not required
data.drop('mes_baja', axis = 1, inplace = True)

# Cyclic Calendar
data['time'] = (2 * pi * (data['epiweek'] - 1) + ((data['diasemana'] - 1) / 7)) / 52
data['sin_time'] = np.sin(data['time'])
data['cos_time'] = np.cos(data['time'])
data.drop(['time', 'diasemana', 'epiweek'], axis = 1, inplace = True)

# 3. Prepare Data for ML
data.drop(['cnae', 'icd9', 'codipostal', 'contracte'], axis = 1, inplace = True)



: 

In [ ]:
data.describe()

: 

In [ ]:
data[(data['ContadorBajasDNI']>25)].count() + data[(data['numtreb']>3200)].count() + data[(data['ContadordiasBajasDNI']>750)].count()

: 

In [ ]:
data.drop(data[(data['ContadorBajasDNI']>25)].index, axis = 0, inplace = True)

: 

In [ ]:
data.describe()

: 

In [ ]:

data.drop(data[(data['numtreb']>3200)].index, axis = 0, inplace = True)
data.drop(data[(data['ContadordiasBajasDNI']>750)].index, axis = 0, inplace = True)

: 

In [ ]:
data

: 

In [ ]:
sample = data.sample(n = 236000, random_state = 8)

: 

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
                        sample.drop('duracion_baja', axis = 1), sample['duracion_baja'], test_size=0.3)

def prepare_data(X_train, X_test):
    # One hot Encode Categorical Features.
    ohe_cols = ['sexo', 'recaida', 'grupcoti', 'pluriempleo',
                'cnae_cat', 'icd9_cat', 'codipostal_cat']
    for col in ohe_cols:
        ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
        ohe.fit(X_train[[col]])
        for X_t in [X_train, X_test]:
            X_t[ohe.get_feature_names_out()] = ohe.transform(X_t[[col]])
            X_t.drop(col, axis = 1, inplace = True)


    # Scale numerical features.
    robust_cols = ['ContadorBajasDNI', 'ContadordiasBajasDNI', 'numtreb', 'tiempo_en_empresa']
    for col in robust_cols:
        rb_scaler = RobustScaler()
        rb_scaler.fit(X_train[[col]])
        for X_t in [X_train, X_test]:
            X_t[col] = rb_scaler.transform(X_t[[col]])

    st_scaler = StandardScaler()
    st_scaler.fit(X_train[['edad']])
    for X_t in [X_train, X_test]:
        X_t['edad'] = st_scaler.transform(X_t[['edad']])

    return X_train, X_test

: 

In [ ]:
X_train_p, X_test_p = prepare_data(X_train, X_test)

: 

In [ ]:
y_log_train = np.log(y_train)
y_log_test = np.log(y_test)

: 

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

: 

In [ ]:
model_LR_1 = LinearRegression()
cv_results_LR_1 = cross_validate(model_LR_1, X_train_p, y_log_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
LR_val_rmse = np.exp(cv_results_LR_1['test_neg_root_mean_squared_error'].mean()*(-1))
LR_val_r2 = cv_results_LR_1['test_r2'].mean()

model_LR_1.fit(X_train_p, y_log_train)
y_pred_LR_1 = model_LR_1.predict(X_test_p)
LR_train_rmse = np.exp(mean_squared_error(y_log_test, y_pred_LR_1, squared = False))

print(f'Validation R2= {np.round(LR_val_r2, 2)}, validation rmse= {np.round(LR_val_rmse, 2)}, and training rmse= {np.round(LR_train_rmse, 2)}')

: 

In [ ]:
from sklearn.linear_model import PoissonRegressor
model_poi_1 = PoissonRegressor(alpha=2, fit_intercept=True, max_iter=1000, tol=0.001)
cv_results_poi_1 = cross_validate(model_poi_1, X_train_p, y_log_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 

poi_val_rmse = np.exp(cv_results_poi_1['test_neg_root_mean_squared_error'].mean()*(-1))
poi_val_r2 = cv_results_poi_1['test_r2'].mean()

model_poi_1.fit(X_train_p, y_log_train)
y_pred_poi_1 = model_poi_1.predict(X_test_p)
poi_train_rmse = np.exp(mean_squared_error(y_log_test, y_pred_poi_1, squared = False))

print(f'Validation R2= {np.round(poi_val_r2, 2)}, validation rmse= {np.round(poi_val_rmse, 2)}, and training rmse= {np.round(poi_train_rmse, 2)}')

: 

In [ ]:
from sklearn.linear_model import SGDRegressor
model_SGD_1 = SGDRegressor(loss='squared_error', alpha = 0.0001, max_iter = 1000, learning_rate = "invscaling", eta0 = 0.01)
cv_results_SGD_1 = cross_validate(model_SGD_1, X_train_p, y_log_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
SGD_val_rmse = np.exp(cv_results_SGD_1['test_neg_root_mean_squared_error'].mean()*(-1))
SGD_val_r2 = cv_results_SGD_1['test_r2'].mean()

model_SGD_1.fit(X_train_p, y_log_train)
y_pred_SGD_1 = model_SGD_1.predict(X_test_p)
SGD_train_rmse = np.exp(mean_squared_error(y_log_test, y_pred_SGD_1, squared = False))

print(f'Validation R2= {np.round(SGD_val_r2, 2)}, validation rmse= {np.round(SGD_val_rmse, 2)}, and training rmse= {np.round(SGD_train_rmse, 2)}')

: 

In [ ]:
from sklearn.svm import SVR
model_SVR_1 = SVR(epsilon=0.1, C=1, kernel='rbf', max_iter=1000)
cv_results_SVR_1 = cross_validate(model_SVR_1, X_train_p, y_log_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
SVR_val_rmse = np.exp(cv_results_SVR_1['test_neg_root_mean_squared_error'].mean()*(-1))
SVR_val_r2 = cv_results_SVR_1['test_r2'].mean()

model_SVR_1.fit(X_train_p, y_log_train)
y_pred_SVR_1 = model_SVR_1.predict(X_test_p)
SVR_train_rmse = np.exp(mean_squared_error(y_log_test, y_pred_SVR_1, squared = False))

print(f'Validation R2= {np.round(SVR_val_r2, 2)}, validation rmse= {np.round(SVR_val_rmse, 2)}, and training rmse= {np.round(SVR_train_rmse, 2)}')


: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_RF_1 = RandomForestRegressor(n_estimators=  1500, min_samples_leaf= 4, min_samples_split= 10)
cv_results_RF_1 = cross_validate(model_RF_1, X_train_p, y_log_train, cv=2,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
RF_val_rmse = np.exp(cv_results_RF_1['test_neg_root_mean_squared_error'].mean()*(-1))
RF_val_r2 = cv_results_RF_1['test_r2'].mean()

model_RF_1.fit(X_train_p, y_log_train)
y_pred_RF = model_RF_1.predict(X_test_p)
RF_train_rmse = np.exp(mean_squared_error(y_log_test, y_pred_RF, squared = False))

print(f'Validation R2= {np.round(RF_val_r2, 2)}, validation rmse= {np.round(RF_val_rmse, 2)}, and training rmse= {np.round(RF_train_rmse, 2)}')

: 

In [ ]:
import xgboost as xgb
model_xgb_1 = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 10, seed = 123)

: 

In [ ]:
from sklearn.model_selection import cross_val_score
XGB_val_r2 = cross_val_score(model_xgb_1, X_train_p, y_log_train, scoring='r2', cv=5).mean()
XGB_val_rmse = cross_val_score(model_xgb_1, X_train_p, y_log_train, scoring='neg_root_mean_squared_error', cv=5).mean()*(-1)

model_xgb_1.fit(X_train_p, y_log_train)
y_pred_xgb = model_xgb_1.predict(X_test_p)
XGB_train_rmse = np.exp(mean_squared_error(y_log_test, y_pred_xgb, squared = False))

print(f'Validation R2= {np.round(XGB_val_r2, 2)}, validation rmse= {np.round(XGB_val_rmse, 2)}, and training rmse= {np.round(XGB_train_rmse, 2)}')

: 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

: 

In [ ]:
model_KNR_1 = KNeighborsRegressor(n_neighbors = 20, leaf_size = 40)
cv_results_KNR_1 = cross_validate(model_KNR_1, X_train_p, y_log_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
)

KNR_val_rmse = np.exp(cv_results_KNR_1['test_neg_root_mean_squared_error'].mean()*(-1))
KNR_val_r2 = cv_results_KNR_1['test_r2'].mean()

model_KNR_1.fit(X_train_p, y_log_train)
y_pred_KNR_1 = model_KNR_1.predict(X_test_p)
KNR_train_rmse = np.exp(mean_squared_error(y_log_test, y_pred_KNR_1, squared = False))

print(f'Validation R2= {np.round(KNR_val_r2, 2)}, validation rmse= {np.round(KNR_val_rmse, 2)}, and training rmse= {np.round(KNR_train_rmse, 2)}')

: 

In [ ]:
from sklearn.linear_model import Ridge, Lasso

: 

In [ ]:
model_ridge_1 = Ridge()
cv_results_ridge_1 = cross_validate(model_ridge_1, X_train_p, y_log_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
pd.DataFrame(cv_results_ridge_1)
ridge_val_rmse = np.exp(cv_results_ridge_1['test_neg_root_mean_squared_error'].mean()*(-1))
ridge_val_r2 = cv_results_ridge_1['test_r2'].mean()

model_ridge_1.fit(X_train_p, y_log_train)
y_pred_ridge_1 = model_ridge_1.predict(X_test_p)
ridge_train_rmse = np.exp(mean_squared_error(y_log_test, y_pred_ridge_1, squared = False))

print(f'Validation R2= {np.round(ridge_val_r2, 2)}, validation rmse= {np.round(ridge_val_rmse, 2)}, and training rmse= {np.round(ridge_train_rmse, 2)}')

: 

In [ ]:
model_lasso_1 = Lasso()
cv_results_lasso_1 = cross_validate(model_lasso_1, X_train_p, y_log_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
lasso_val_rmse = np.exp(cv_results_lasso_1['test_neg_root_mean_squared_error'].mean()*(-1))
lasso_val_r2 = cv_results_lasso_1['test_r2'].mean()

model_lasso_1.fit(X_train_p, y_log_train)
y_pred_lasso_1 = model_lasso_1.predict(X_test_p)
lasso_train_rmse = np.exp(mean_squared_error(y_log_test, y_pred_lasso_1, squared = False))

print(f'Validation R2= {np.round(lasso_val_r2, 2)}, validation rmse= {np.round(lasso_val_rmse, 2)}, and training rmse= {np.round(lasso_train_rmse, 2)}')

: 

In [ ]:
from sklearn.linear_model import ElasticNet

: 

In [ ]:
model_EN_1 = ElasticNet()
cv_results_EN_1 = cross_validate(model_EN_1, X_train_p, y_log_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
pd.DataFrame(cv_results_EN_1)
val_EN_rmse = np.exp(cv_results_lasso_1['test_neg_root_mean_squared_error'].mean()*(-1))
val_EN_r2 = cv_results_lasso_1['test_r2'].mean()

model_EN_1.fit(X_train_p, y_log_train)
y_pred_EN_1 = model_EN_1.predict(X_test_p)
pred_EN_rmse = np.exp(mean_squared_error(y_log_test, y_pred_EN_1, squared = False))

print(f'Validation R2= {np.round(lasso_val_r2, 2)}, validation rmse= {np.round(val_EN_rmse, 2)}, and training rmse= {np.round(pred_EN_rmse, 2)}')

: 

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model_DTR_1 = DecisionTreeRegressor()
cv_results_DTR_1 = cross_validate(model_DTR_1, X_train_p, y_log_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
val_DTR_rmse = np.exp(cv_results_DTR_1['test_neg_root_mean_squared_error'].mean()*(-1))
val_DTR_r2 = cv_results_DTR_1['test_r2'].mean()
model_DTR_1.fit(X_train_p, y_log_train)
y_pred_DTR_1 = model_DTR_1.predict(X_test_p)
pred_DTR_rmse = np.exp(mean_squared_error(y_log_test, y_pred_DTR_1, squared = False))

print(f'Validation R2= {np.round(lasso_val_r2, 2)}, validation rmse= {np.round(val_DTR_rmse, 2)}, and training rmse= {np.round(pred_DTR_rmse, 2)}')

: 

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

: 

In [ ]:
model = models.Sequential()
model.add(layers.Dense(10, activation='relu', input_dim=116))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dropout(rate=0.2))
model.add(layers.Dense(1, activation='linear'))
model.summary()

: 

In [ ]:
model.compile(loss='mse', optimizer = 'adam', metrics='RootMeanSquaredError')

: 

In [ ]:
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train_p, y_log_train, validation_split=0.3, callbacks = [es], epochs = 100)

: 

In [ ]:
res = model.evaluate(X_test_p, y_log_test)[1]

: 

In [ ]:
y_pred_NN = model.predict(X_test_p)

: 

In [ ]:
np.exp(mean_squared_error(y_log_test, y_pred_NN, squared = False))

: 

In [ ]:
from sklearn.model_selection import GridSearchCV # Train/Test split

model_RF = RandomForestRegressor()
# Hyperparameter Grid
RF_grid = {'n_estimators': [600, 1500],
        'min_samples_split': [10, 15, 50],
        'min_samples_leaf': [4, 6]}
# Instanciate Grid Search
RF_search = GridSearchCV(model_RF, RF_grid,
                           scoring = 'r2',
                           cv = 3,
                           n_jobs=-1 # paralellize computation
                           )

# Fit data to Grid Search
RF_search.fit(X_train_p, y_log_train)

print(RF_search.best_score_)
print(RF_search.best_params_)

: 

In [ ]:
search.best_score_

: 

In [ ]:
search.best_params_

: 

In [ ]:
model_XGB_2 = xgb.XGBRegressor()
parameters_XGB = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [0.01, 0.03], #so called `eta` value
              'max_depth': [7, 8, 10],
              'min_child_weight': [4, 6],
              'subsample': [0.7, 0.9],
              'colsample_bytree': [0.7],
              'n_estimators': [500, 750]}

xgb_grid = GridSearchCV(model_XGB_2,
                        parameters_XGB,
                        scoring = 'r2',
                        cv = 5,
                        n_jobs = -1,
                        verbose=True)

xgb_grid.fit(X_train_p, y_log_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

: 

: 